In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from itertools import permutations 



## Predicción de Juegos 

Un club del juego de Go recopiló los resultados de varias partidas entre diferentes jugadores, almacenados en el archivo juegos_entrenamiento.txt, con el objetivo de predecir el resultado de partidas futuras, ejemplos de las cuales se encuentran en el archivo *juegos_validacion.txt*. 

*archivos juegos_entrenamiento.txt* y *juegos_validacion.txt2* contienen 3 columnas: la primera corresponde al identificador del jugador A, la segunda al identificador del jugador B y la tercera es el resultado de la partida (1 si ganó el jugador A o 0 si ganó el jugador B). En el club hay un total de D jugadores, por lo que cada identificador es un número entero entre 1 y D. La predicción del resultado de un juego se puede plantear como un problema de clasificación: dados 2 jugadores (A y B) se requiere predecir si A ganó (y = 1) o si fue B (y = 0). Realice los siguientes ejercicios:

* Entrena y evalúa un clasificador bayesiano ingenuo. Al ser un modelo generativo (modelala probabilidad conjunta $P(x, y)$), es posible generar partidas artificiales con los parámetros calculados. Genera nuevas partidas que sigan la distribución modelada

In [2]:
# Lectura de datos

x_train = np.loadtxt("regl_data/juegos_entrenamiento.txt")
x_val = np.loadtxt("regl_data/juegos_validacion.txt")
 
d = np.shape(x_train)[0]    # Numero de jugadores 

nG = sum(x_train[:,-1:]==1) # Numero de Ganadores 
nP = sum(x_train[:,-1:]==0) # Numero de Perdedores  

#nPA = nGB # Numero de Perdedores A
#nPB = nGA # Numero de Perdedores B

#encuentros que gano A
JugGanA = x_train[x_train[:,-1]==1][:,0] #ID de ganadores de A
nGA = len(JugGanA)
JugPerB = x_train[x_train[:,-1]==1][:,1] #ID de perdedores de B
nPB = len(JugPerB)
#encuentros que gano B
JugGanB = x_train[x_train[:,-1]==0][:,0] #ID de ganadores de A
nGB = len(JugGanB)
JugPerA = x_train[x_train[:,-1]==0][:,1] #ID de perdedores de B
nPA = len(JugPerA)

Vamos a considerar que los jugadores A y B, siguen una función de distribución categórica, dada por 

$$f(x;\vec{q}) = \prod_{k=1}^{d}q_{k}^{[x = k]}$$

Donde los estimadores de la función categórica están dados por la siguiente expresión, considerando que vamos a utilizar MAP

$$\hat{q}_{k} = \frac{c_{k}+\alpha_{k}-1}{n+\sum_{k=1}^{K}\alpha_{k}+K}$$

Luego se plantea el clasificador como sigue

$$C= \underset{C\in\{Ganar,Perder\}}{\mathrm{ArgMax}} \left[ P(C)\prod_{k=1}^{d}{(q_{A})}_{C}\prod_{k=1}^{d}{(q_{B})}_{C}\right] $$

Donde la clase se distribuye de manera binomial , ahora se calculan los respectivos constantes



In [5]:
# Funcion que implementa el clasificador utilizando estimadores de maximo a posteriori

def MAP(x,Alpha):
    # Eleccion de alpha para tenerlo como entrada
    K = 4
    alpha1,alpha2,alpha3,alpha4 = Alpha[0],Alpha[1],Alpha[2],Alpha[3]
    s1,s2,s3,s4 = alpha1*K*(K+1),alpha2*K*(K+1),alpha3*K*(K+1),alpha4*K*(K+1)
    # Probabilidades de la clase
    PA = nG/d ; PB = nP/d 
    
    # Se identifica cuantas veces juega (Gana/Pierde) cada jugador  
    qGA = np.unique(JugGanA,return_counts = True) 
    qPB = np.unique(JugPerB,return_counts = True)

    qGB = np.unique(JugGanB,return_counts = True) 
    qPA = np.unique(JugPerA,return_counts = True)

    # Diccionario que hace el conteo de cada jugador y lo relaciona con el numero de jugador
    QGA = {qGA[0][i]:qGA[1][i] for i in range(len(qGA[0]))}
    QPB = {qPB[0][i]:qPB[1][i] for i in range(len(qPB[0]))}

    QGB = {qGB[0][i]:qGB[1][i] for i in range(len(qGB[0]))}
    QPA = {qPA[0][i]:qPA[1][i] for i in range(len(qPA[0]))}
    
    # Implementación de la definición se debe aplicar dado que consideramos un diccionario
    if x[0] in QGA and x[1] in QGB:  CA = float(PA*((QGA[x[0]]+alpha1-1)/(nGA+s1-K))*((QGB[x[1]]+alpha2-1)/(nGB+s2-K)))
    if x[0] in QGA and x[1] not in QGB:  CA = float(PA*((QGA[x[0]]+alpha1-1)/(nGA+s1-K))*(alpha2-1)/(nGB+s2-K))
    if x[0] not in QGA and x[1] in QGB: CA = float(PA*(alpha1-1/(nGA+s1-K))*((QGB[x[1]]+alpha2-1)/(nGB+s2-K)))
    if x[0] not in QGA and x[1] not in QGB: CA = 0

    if x[0] in QPA and x[1] in QPB:  CB = float(PB*((QPA[x[0]]+alpha3-1)/(nPA+s3-K))*((QPB[x[1]]+alpha4-1)/(nPB+s4-K)))
    if x[0] in QPA and x[1] not in QPB:  CB = float(PB*((QPA[x[0]]+alpha3-1)/(nPA+s3-K))*(alpha4-1)/(nPB+s4-K))
    if x[0] not in QPA and x[1] in QPB: CB = float(PB*(alpha3-1/(nPA+s3-K))*((QPB[x[1]]+alpha4-1)/(nPB+s4-K)))
    if x[0] not in QPA and x[1] not in QPB: CB = 0
    
    clases = {float(CA):1,float(CB):0}
        
    return clases[max(CA,CB)]


# Definicion de un score , verificando cuantas veces adivina el clasificador con el conjunto
# de validacion 

def score(x_train,x_val):
    s = 0
    for i in range(len(x_val)):
        s+=MAP(x_train[:,[0,1]][i],[2,2,0,1])==x_val[:,-1][i]
    return s/len(x_val)

# para esta eleccion particular de alpha, se tiene este score
print("score del modelo = {:.3f}".format(score(x_train,x_val)))

score del modelo = 0.617


Ahora se juega un poco con los valores de $\alpha$ para esto hacemos grupos de 4 teniendo en cuenta los numeros de 1 a 10 para encontrar en esta pequeña prueba el valor que maximiza nuestra predicción

In [9]:
perm = list(permutations(np.arange(10), 4))

#buscamos unos valores un poco mejores para el modelo
S = []
for j in perm:
    s = 0
    for i in range(len(x_val)):
        s+=MAP(x_train[:,[0,1]][i],j)==x_val[:,-1][i]
    S.append([s/len(x_val),j])
S = np.array(S)

S[np.argmax(S[:,0])]

array([0.6521739130434783, (9, 3, 0, 1)], dtype=object)

* Entrena y evalúa un clasificador de regresión logística. Para esto es necesario reparametrizar las entradas. Explica el procedimiento y la lógica de la reparametrización que realizaste. La selecciona y visualiza los valores de los parámetros.Grafica las curvas ROC y de precisiónexhaustividad y reporta sus áreas bajo la curva.

Hacemos la reparametrización con variables dummy para cunatificar las variables categóricas del modelo 

In [8]:
JugadorA = x_train[:,0]
JugadorB = x_train[:,1]
t = x_train[:,-1].reshape(len(JugadorA),1)

N = len(JugadorA)
#funcion que produce una matriz con las variables dummy de una característica 
def dummy_matrix(x):
    J = np.unique(x)
    N = len(J)
    M = np.zeros((len(x),N-1))
    for i in range(N-1):
        index = np.where(x==J[i])
        M[:,i][index] = np.ones(len(index[0]))
    M[-1] = 0
    return M

# Se armman las matrices que contienen las variables dummy para cada caracteristica, sin embargo son menos los jugadores de B que los que hay en A
# por tanto debemos armar la matriz con el tamaño de jugadores que hay en A

NumJA = len(np.unique(JugadorA))
NumJB = len(np.unique(JugadorB))

# Se considera una matriz cuadrada de ceros de tamaño del que más jugadores tenga, para empezar a armar la matriz de variables
unos = np.ones((1,N)).T

#Ahora se deben rellenar con las variables Dummy, por definicion tiene el tamaño de una de las matrices dummy, por lo cual no debería tener problemas con la primera
#caracteristica. El problema esta con la segunda característica, vamos e rellenar con ceros lo que le hace falta para tener el mismo tamaño que la matriz de la 
#primera característica

#Calculamos las matrices dummy
DA = dummy_matrix(JugadorA)
DB = dummy_matrix(JugadorB)



0.0

Ya se hizo la reparametrización de las variables categóricas como variables dummy, ahora se debe considerar la regresión logistica. Vemos que para dos clases, la probabilidad posterior están dadas por una función Sigmoide, dadas por 

$$P(C_{1}|\phi) = y(\phi) = \sigma(\vec{\omega}^{T}\vec{\phi})$$

Se usa la máxima verosimilitud para determinar los parametros $\vec{\omega}$ de este modelo directamente.

Ahora se escribe la función de probabilidad usando un esquema $1-K$ con el vector objetivo $\vec{t}_{n}$ para un vector de características $\vec{\phi}_{n}$ perteneciente a la clase  

$$P(t|w) = \prod_{n=1}^{N}y_{n}^{t_{n}}(1-y_{n})^{1-t_{n}}$$

Donde $\vec{t} = (t_{1},\cdots,t_{N})$ y $y_{n} = P(C_{1}|\phi_{n})$


Tomando el logaritmo negativo se tiene la llamada entropía-cruzada, que es la función de error para la clasificación es

$$E(\vec{\omega})=-\ln P(\vec{t_{1}},\cdots,\vec{t_{N}}|\vec{\omega})=-\sum_{n=1}^{N}[t_{n}\ln y{n} +(1-t_{n})\ln (1-y_{n})]$$

Ahora el gradiente de esta expresión se puede deducir de la siguiente manera donde 


$$\nabla_{\vec{\omega}}E(\vec{w})=\sum_{n=1}^{N}(y_{n}-t_{n})\vec{\phi}$$

Ahora se necesita una manera de encontrar la actualización del vector de parámetros, como $E(\vec{\omega})$ es convex se puede asegurar que la función tiene un minimo global único, para encontrar el paso de actualización dse utiliza el método de Newton-Raphson para minimizar cualquier función está dado por 

$$\vec{\omega}^{\tau+1} = \vec{\omega}^{\tau} - \mathcal{H}^{-1}\nabla_{\omega}E(\vec{\omega})$$

Donde $\mathcal{H}$ es la matriz Hessiana compuesta de las segundas derivadas de la función de error, que por definición está dada por 

$$\mathcal{H} = \nabla_{\omega}\nabla_{\omega}E(\vec{\omega}) = \sum_{n=1}^{N}y_{n}(1-y_{n})\phi_{n}\phi_{n^{T}}$$

notamos que $\mathcal{H}$ depende del estado actual de $\vec{\omega}^{T}$ 

Usando la notación de matriz encontramos las cantidades que tenemos que encontrar, dadas por

$$\nabla_{\vec{\omega}}E(\vec{w}) = \Phi^{T}(\vec{y}-\vec{t})$$

donde $\Phi$ es la matriz de diseño $N\times M$, $\vec{y}$ es el vector de salida , que utiliza la función sigmoide y es similar al vector $\vec{t}$ que es el vector de objetivos del sistema

La matriz $\mathcal{H}$ puede ser escrita como sigue 

$$\mathcal{H} = \Phi^{T}\mathcal{R}\Phi$$

donde $\mathcal{R}$ es una matriz diagonal de $N\times N$ con elementos 

$$\mathcal_{R}_{nn} = y{n}(1-y_{n})$$

se interpreta como la matriz 


In [11]:
# ahora armamos la matriz que representa las variables dummy 
D = np.concatenate((DA,DB),axis=1)

#Definamos la funcion sigmoide
def sigmoid(z):
    return 1/(1+np.exp(-z))

#Construimos la matriz de diseño
unos = np.ones((np.shape(D)[0],1))
Phi = np.concatenate((unos,D),axis = 1)

# debemos encontrar parametros de la linealizacion creamos un vector de numeros aleatorios
# que corresponda con el numero de variables 
w = np.random.uniform(0,0.001,(1,np.shape(Phi)[1])).T

#calculamos y
Y = sigmoid(w.T@Phi.T).T

#Calculamos el gradiente
G = Phi.T@(Y-t)

# calculamos la matriz Hessiana, pero primero se construye la matriz de covarianza
r = np.zeros((len(JugadorA),len(JugadorA)))
np.fill_diagonal(r,Y*(1-Y))
H = Phi.T@r@Phi

# Luego la actualización por Neewton-Rhapson

np.linalg.det(H[0:150,0:150])

8.66442736061291e-99

* Compara el clasificador bayesiano ingenuo y regresión logística en este problema. ¿Qué ventajas y desventajas tienen los modelos entrenados? ¿Qué pasaría si se entrena el clasificador bayesiano ingenuo con los vectores reparametrizados o si se entrena un modelo de regresión logística usando los vectores de entrada originales? ¿Consideras que las presuposiciones de cada clasificador son apropiadas para los datos del problema? ¿Para este tipo de problemas cuál de los dos recomendarías y por qué?

* Deriva la regla de actualización para el algoritmo del descenso por gradiente de un clasificador donde $\hat{y} = \text{sigm}(\vec{\theta}^{T}\vec{x})$ y la función de pérdida sea

$$E(\vec{\theta}) = \frac{1}{2}\sum_{i=1}^{n}(\hat{y}^{(i)}-y^{(i)})$$

  Discute las diferencias entre este clasificador y el de regresión logística y compara sus rendimientos en la tarea de predicción de juegos.